In [28]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

df1 = pd.read_excel('/Users/seanyu_mac/Downloads/report (7).xlsx')
df2 = pd.read_excel('/Users/seanyu_mac/Downloads/report (8).xlsx')

df = pd.concat([df1, df2])
original_df = df

con1 = df['주문 상태'] == '배송완료'
con2 = df['주문 상태'] == '구매확정'
con3 = df['주문 상태'] == '배송중'
con4 = df['주문 상태'] == '배송준비'
con5 = df['주문 상태'] == '결제완료'

df = df[con1 | con2 | con3 | con4 | con5]

df['order_id'] = df['주문 번호']
df['date'] = df['결제 완료일'].dt.strftime('%Y-%m-%d')
df['product'] = df['상품명']
df['sales'] = df['정산가']
df['option'] = df['옵션명']
df['name'] = df['구매자명']

df = df[['order_id', 'date', 'product', 'option', 'name','sales']]
df['date'] = pd.to_datetime(df['date'])

sales_df = pd.DataFrame(df.groupby('date')['sales'].sum())
sales_df['date'] = sales_df.index
sales_df = sales_df[['date', 'sales']]
sales_df.reset_index(drop=True, inplace=True)

product_df = pd.DataFrame(df.groupby('product')['sales'].sum())
product_df['product'] = product_df.index
product_df = product_df[['product', 'sales']]
product_df.reset_index(drop=True, inplace=True)

with pd.ExcelWriter('/Users/seanyu_mac/Downloads/친한스토어매출.xlsx') as writer:
    sales_df.to_excel(writer, sheet_name='일자별', index=False)
    product_df.to_excel(writer, sheet_name='상품별', index=False)

total_sales = np.round(df['sales'].sum(), 1)
total_num = len(df['order_id'].unique())
daily_sales = np.round(total_sales / len(df['date'].unique()), 1)
arpu = np.round(total_sales / len(df['name'].unique()), 1)
con1 = original_df['주문 상태'] == '취소완료'
con2 = original_df['주문 상태'] == '판매자취소'
cancel_rate = np.round(original_df[con1 | con2].shape[0] / original_df.shape[0] * 100, 1)

print(f'누적 매출액 : {total_sales:,}원 / 누적 판매건수 : {total_num:,}건 / 일평균 매출액 : {daily_sales:,}원 / 객단가 : {arpu:,}원 / 구매취소율 : {cancel_rate}%')

누적 매출액 : 5,861,540원 / 누적 판매건수 : 151건 / 일평균 매출액 : 142,964.4원 / 객단가 : 65,128.2원 / 구매취소율 : 16.2%


/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/var/folders/wl/txjbcgld2sngchsc7txxqppm0000gp/T/ipykernel_39068/127966725.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['order_id'] = df['주문 번호']
/var/folders/wl/txjbcgld2sngchsc7txxqppm0000gp/T/ipykernel_39068/127966725.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [14]:
df['month'] = df['date'].dt.strftime('%m')
df.groupby('month')['sales'].sum()

month
09     386850
10    4214330
11    1186960
Name: sales, dtype: int64

In [39]:
con1 = original_df['결제 완료일'] >= datetime(2024, 10, 1)
con2 = original_df['결제 완료일'] < datetime(2024, 11, 1)
original_df[con1 & con2].index.size

117